# Angleichung der Label Bezeichnungnen beider Datensätze

In [1]:
import os 
import pandas as pd
import logging
import multiprocessing as mp

In [2]:
# Logging Parameter
logging.basicConfig(
    filename='22_angleichung_labels.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Verzeichnis der original CSV-Dateien
verzeichnis_ids17 = '../01_Datensaetze/improved_cic-ids-2017'
verzeichnis_ids18 = '../01_Datensaetze/improved_cse-cic-ids-2018'
verzeichnis_ids17_processed = '../01_Datensaetze/improved_cic-ids-2017/processed'
verzeichnis_ids18_processed = '../01_Datensaetze/improved_cse-cic-ids-2018/processed'

nrows = None
chunksize = 1000000

# Mapping von CICIDS2017-Bezeichnungen zu CICIDS2018-Bezeichnungen
mapping_2017_to_2018 = {
    'Botnet': 'Botnet Ares',
    'Botnet - Attempted': 'Botnet Ares - Attempted',
    'DDoS': 'DDoS-LOIC-HTTP',
    'Infiltration - Portscan': 'Infiltration - NMAP Portscan',
    'SSH-Patator': 'SSH-Bruteforce'
}

In [3]:
def get_labels_from_file(verzeichnis, datei):
    if datei.endswith('.csv'):
        pfad_zur_datei = os.path.join(verzeichnis, datei)
        try:
            df = pd.read_csv(pfad_zur_datei, nrows=nrows)
            found_labels = df['Label'].unique()
            logging.info(f"Successfully read {pfad_zur_datei}. Found labels: {found_labels}")
            return found_labels
        except Exception as e:
            logging.error(f"Error reading {pfad_zur_datei}: {e}")
            return []
    return []

def get_labels_from_dir(verzeichnis):
    try:
        logging.info(f"Start processing directory {verzeichnis}")
        with mp.Pool(processes=2) as pool:
            results = pool.starmap(get_labels_from_file, [(verzeichnis, datei) for datei in os.listdir(verzeichnis)])

        labels = set()
        for result in results:
            labels.update(result)
        
        logging.info(f"Successfully processed directory {verzeichnis}")
        return labels
    except Exception as e:
        logging.error(f"Error processing directory {verzeichnis}: {e}")
        return set()

In [4]:
labels_ids17 = get_labels_from_dir(verzeichnis_ids17)

KeyboardInterrupt: 

In [6]:
labels_ids18 = get_labels_from_dir(verzeichnis_ids18)

In [10]:
labels_ids17

{'BENIGN',
 'Botnet Ares',
 'Botnet Ares - Attempted',
 'DDoS-LOIC-HTTP',
 'DoS GoldenEye',
 'DoS GoldenEye - Attempted',
 'DoS Hulk',
 'DoS Hulk - Attempted',
 'DoS Slowloris',
 'DoS Slowloris - Attempted',
 'FTP-Patator - Attempted',
 'Infiltration - NMAP Portscan',
 'SSH-Bruteforce',
 'Web Attack - Brute Force',
 'Web Attack - Brute Force - Attempted',
 'Web Attack - SQL Injection',
 'Web Attack - SQL Injection - Attempted',
 'Web Attack - XSS',
 'Web Attack - XSS - Attempted'}

In [ ]:
labels_ids18

In [ ]:
labels_ids17 == labels_ids18

Die Angriffe unterscheiden sich teilweise. Nach genauer Analyse werden die Datensätze aneinander angepasst. Dabei werden Angriffe entfernt, zusammengefasst oder Bezeichnungen angepasst. 


## Anpassung der Bezeichnungen der Labels

In [17]:
# Funktion, um eine einzelne CSV-Datei zu verarbeiten und Labels zu ersetzen
def relabel_file(file_path):
    try:
        # Lade die CSV-Datei in Chunks
        chunks = pd.read_csv(file_path, chunksize=chunksize)

        # Verarbeite jeden Chunk
        for chunk in chunks:
            # Prüfe, ob es eine Spalte mit den Angriffsnamen gibt (z.B. 'Label')
            if 'Label' in chunk.columns:
                # Ersetze die Labels basierend auf dem Mapping
                chunk['Label'] = chunk['Label'].map(mapping_2017_to_2018).fillna(chunk['Label'])

                # Speichere den Chunk mit den geänderten Labels zurück in die Originaldatei
                chunk.to_csv(file_path, mode='a', header=False, index=False)
            else:
                logging.warning(f"Keine 'Label'-Spalte gefunden in {file_path}")
        
        logging.info(f"Datei verarbeitet und gespeichert: {file_path}")
    
    except Exception as e:
        logging.error(f"Fehler beim Verarbeiten der Datei {file_path}: {e}")

# Funktion, um alle CSV-Dateien im Verzeichnis zu finden und zu verarbeiten
def process_all_files_in_directory(directory):
    # Hole alle CSV-Dateien im Verzeichnis
    csv_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]

    # Nutze Multiprocessing, um die Dateien parallel zu verarbeiten
    with mp.Pool(processes=4) as pool:
        pool.map(relabel_file, csv_files)


In [18]:
process_all_files_in_directory(verzeichnis_ids17) #Bezeichnungen in den Dateien von CICIDS2017 werden an die Bezeichnungen von CICIDS2018 angepasst

KeyboardInterrupt: 

In [13]:
new_labels_ids17 = get_labels_from_dir(verzeichnis_ids17)

Die Bezeichnungen sind angepasst.

## Angriffe entfernen, die nicht in beiden Datensätzen vorhanden sind.

In [14]:
remove_in_ids17 = ['DoS Slowhttptest', 'DoS Slowhttptest - Attempted', 'FTP-Patator', 'Heartbleed', 'Infiltration', 'Infiltration - Attempted', 'Portscan', 'SSH-Patator - Attempted']
remove_in_ids18 = ['DDoS-HOIC', 'DDoS-LOIC-UDP', 'DDoS-LOIC-UDP - Attempted', 'Infiltration - Communication Victim Attacker', 'Infiltration - Dropbox Download', 'Infiltration - Dropbox Download - Attempted']

In [10]:
def remove_rows_with_labels(file_path, labels_to_remove, output_file_path):
    try:
        # Lade die CSV-Datei in Chunks
        chunks = pd.read_csv(file_path, chunksize=chunksize)

        # Verarbeite jeden Chunk
        for chunk in chunks:
            # Entferne die Zeilen mit den angegebenen Labels
            chunk = chunk[~chunk['Label'].isin(labels_to_remove)]

            # Speichere den Chunk ohne die entfernten Zeilen in die neue Datei
            chunk.to_csv(output_file_path, mode='a', header=not os.path.exists(output_file_path), index=False)
        
        logging.info(f"Zeilen mit Labels '{labels_to_remove}' entfernt und Datei gespeichert: {output_file_path}")
    
    except Exception as e:
        logging.error(f"Fehler beim Verarbeiten der Datei {file_path}: {e}")

def remove_labels_from_all_files(directory, labels_to_remove):
    # Hole alle CSV-Dateien im Verzeichnis
    csv_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]

    # Nutze Multiprocessing, um die Dateien parallel zu verarbeiten
    with mp.Pool(processes=4) as pool:
        pool.starmap(remove_rows_with_labels, [(file, labels_to_remove, f"{file}_processed.csv") for file in csv_files])


In [56]:
remove_labels_from_all_files(verzeichnis_ids17, remove_in_ids17)

In [10]:
new_labels_ids17 = get_labels_from_dir(verzeichnis_ids17)

In [15]:
new_labels_ids17

{'BENIGN',
 'Botnet Ares',
 'Botnet Ares - Attempted',
 'DDoS-LOIC-HTTP',
 'DoS GoldenEye',
 'DoS GoldenEye - Attempted',
 'DoS Hulk',
 'DoS Hulk - Attempted',
 'DoS Slowloris',
 'DoS Slowloris - Attempted',
 'FTP-Patator - Attempted',
 'Infiltration - NMAP Portscan',
 'SSH-Bruteforce',
 'Web Attack - Brute Force',
 'Web Attack - Brute Force - Attempted',
 'Web Attack - SQL Injection',
 'Web Attack - SQL Injection - Attempted',
 'Web Attack - XSS',
 'Web Attack - XSS - Attempted'}

In [ ]:
remove_labels_from_all_files(verzeichnis_ids18, remove_in_ids18)

In [ ]:
new_labels_ids18 = get_labels_from_dir(verzeichnis_ids18)